In [34]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util
import json
import requests
from datetime import datetime

In [35]:
scope="playlist-modify-public playlist-modify-private"

In [36]:
credentials = "spotify_keys.json"
with open(credentials, "r") as keys:
    api_tokens = json.load(keys)

In [37]:
client_id = api_tokens["client_id"]
client_secret = api_tokens["client_secret"]
redirectURI = api_tokens["redirect_uri"]
weather_api_key = api_tokens["weather_api_key"]
playlist_id = api_tokens["playlist_id"]
username = api_tokens["username"]

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri=redirect_uri,
    scope="playlist-modify-public playlist-modify-private"
    ))

In [38]:
city = "London"  

weather_url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_api_key}&units=metric"
weather_response = requests.get(weather_url)
weather_data = weather_response.json()

weather_condition = weather_data['weather'][0]['main'].lower()
temperature = weather_data['main']['temp']

In [39]:
weather_music = {
    'thunderstorm': ['rock', 'metal'],
    'rain': ['jazz', 'lofi'],
    'snow': ['classical', 'ambient'],
    'clear': ['pop', 'dance'],
    'clouds': ['indie', 'alternative'],
    'mist': ['ambient', 'downtempo'],
    'drizzle': ['jazz', 'lofi'],
    'fog': ['ambient', 'classical']
}

In [40]:
if weather_condition in weather_music:
    genres = weather_music[weather_condition]
else:
    genres = ['pop']  

In [42]:
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)
sp = spotipy.Spotify(auth=token)

tracks = []
for genre in genres:
    results = sp.recommendations(
        seed_genres=[genre],
        limit=10
    )
    tracks.extend([track['uri'] for track in results['tracks']])

In [43]:
print(tracks)

['spotify:track:1iepnn71MidOJZ7Y3vfgE4', 'spotify:track:12pdXFcruNcPUbZypKUDgz', 'spotify:track:2HnMhvOVA1FK8C61nMhJEj', 'spotify:track:3EytfC22ikoXM921cJbejn', 'spotify:track:0EKT63mcvEXuB5rxCW9pzP', 'spotify:track:2UWdUez9MB9yzL7Y81Mcip', 'spotify:track:3CAX47TnPqTujLIQTw8nwI', 'spotify:track:0ARp9mXXpPflIwehy25kCa', 'spotify:track:2PtBhfoPZ6VYtXkrE5FrCH', 'spotify:track:0idjTHGhGRBdWTmsalq3tF', 'spotify:track:5SrnKN9CdGLLHIOUQMHNod', 'spotify:track:2keKJ9KWoGpucAqZ9W4CFi', 'spotify:track:1JdKrFyoU05abww0Zv0ayQ', 'spotify:track:2sCoROOlNQyFpRQEe6A5lv', 'spotify:track:2zrIyIdqafYozfsgiMY0Ae', 'spotify:track:1e5grnGb7YIcX9EuJM1mtB', 'spotify:track:2xwedMsRwKEtLos3oN5pt9', 'spotify:track:0ucps9UAU7I7MmTbnk2YaK', 'spotify:track:1ehPiY9lYnZAA9ogCnBf9G', 'spotify:track:1tT1tIZAINzFZyjoPKtg7C']


In [44]:
tracks = list(set(tracks))[:20]

sp.playlist_replace_items(playlist_id, tracks)

playlist_info = f"Weather-based playlist for {city}. Weather: {weather_condition}. Updated: {datetime.now().strftime('%Y-%m-%d %H:%M')}"
sp.playlist_change_details(playlist_id, description=playlist_info)

In [45]:
print(f"Weather Condition: {weather_condition}")
print(f"Selected Genres: {genres}")
print(f"Number of tracks added: {len(tracks)}")

Weather Condition: clouds
Selected Genres: ['indie', 'alternative']
Number of tracks added: 20


In [47]:
playlist_tracks = sp.playlist_tracks(playlist_id)
print("\nAdded Tracks:")
for idx, item in enumerate(playlist_tracks['items'], 1):
    track = item['track']
    print(f"{idx}. {track['name']} - {track['artists'][0]['name']}")


Added Tracks:
1. Call Your Girlfriend - Robyn
2. Vasoline - Stone Temple Pilots
3. Supermassive Black Hole - Muse
4. That's What You Get - Paramore
5. Super Natural - Turnover
6. The Gold - Manchester Orchestra
7. Higher - Creed
8. People Change - Mipso
9. Radio - Sylvan Esso
10. The Suburbs - Arcade Fire
11. Ash Tree Lane - MS MR
12. Reflections - MisterWives
13. Caring Is Creepy - The Shins
14. Big Sis - SALES
15. Dammit - blink-182
16. Love Is Mystical - Cold War Kids
17. Old Pine - Ben Howard
18. Sleepwalker - Moon Duo
19. Blue Cassette - Friendly Fires
20. Reunion - M83
